In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

<a id="notebook_id"></a>
# `ArrayList`-based stack

<div class="alert alert-block alert-success"> 
    Source code for this notebook can be found in the package <tt>ca.queensu.cisc124.notes.generics.basics</tt>.
</div>

In the previous notebook we saw two approaches to creating stacks of different types of elements:

1. create a different stack class for each element type
    * while easy to do, this has significant disadvantages:
        * there is substantial code duplication
        * we cannot pass instances of the different stack classes to methods using the `Stack` interface because the interface describes a stack of strings
2. create a stack class where the element type is `Object`
    * this approach does allow for stacks of different element types but has significant disadvantages:
        * the programmer must cast the return value from `pop` because an `Object` reference is returned
        * there is no type safety because any element type can be pushed onto a stack

In the first approach, we saw that the type of element stored in the stack has almost no influence on the implementation of the stack class because we never need to invoke a method
or constructor using the element type. In other words, the type of element stored in the stack is an implementation detail that we would like to abstract away. If Java had a mechanism 
that allowed the programmer to store the element type in a variable, then we could store the element type in a variable
and use the variable whenever we needed to refer to the element type. This is precisely what Java's generics facility allows the programmer to do.

Java's generics facility allows the programmer to create classes, enumerations, interfaces, and methods having one or more *type variables* (also called *type parameters*). 
A type variable is a name that is used as a placeholder for an actual type. For example, `List<E>` is a generic interface and `ArrayList<E>` is a generic class both having the type variable `E`.
Generics allow the programmer to generalize a class or interface by allowing types to become parameters of the class or interface.



### Using a generic type

Before seeing how to create a generic class or interface, it is worth briefly reviewing how to use a generic type.
To use a generic class or method, the programmer specifies an actual type for each of the type variables.. For example, a list-of-strings can be created by specifying the 
type `String` for the type variable:

In [ ]:
import java.util.List;
import java.util.ArrayList;

List<String> t = new ArrayList<String>();    // type variable is String

The previous example can be simplified slightly in Java versions 7 and beyond by taking advantage of *generic type inference*:

> You can replace the type arguments required to invoke the constructor of a generic class with an empty set of type parameters (`<>`) 
> as long as the compiler can infer the type arguments from the context. This pair of angle brackets is informally called *the diamond*.
>
> [Type Inference and Instantiation of Generic Classes](https://docs.oracle.com/javase/tutorial/java/generics/genTypeInference.html#type-inference-instantiation)

Because `t` is declared to be a list-of-string, the type variable can be omitted from the constructor call:

In [ ]:
import java.util.List;
import java.util.ArrayList;

List<String> t = new ArrayList<>();    // no type variable needed to call the ArrayList constructor
                                       // because t is declared to be List<String>

<div class="alert alert-info">

**Type variables must be reference types:** A limitation of Java generics is that type variables must be reference types; primitive types are not allowed as type variables.
The reason for this is that generics were added to the Java language some time after the language had been in wide use, and allowing primitive types as type variables would
have required existing code to have to be re-compiled or modified. The details of the design choices behind Java generics and a possible approach for allowing primitive types
    as type variables are described in the [following article](http://cr.openjdk.java.net/~briangoetz/valhalla/specialization.html#design-choices-behind-java-generics).

</div>

## Implementing a generic stack interface and class

To create a generic stack type, we need to add a type variable to the interface and class declarations. While any Java identifier can be used for a type variable name, the convention is to
usually use a single capitalized letter for the type variable name. 

> This stands in sharp contrast to the variable naming conventions that you already know about, and with good reason: Without this convention, it would be difficult to tell the
> difference between a type variable and an ordinary class or interface name.
>
> The most commonly used type parameter names are:
> 
> * E - Element (used extensively by the Java Collections Framework)
> * K - Key
> * N - Number
> * T - Type
> * V - Value
>
> [Type Parameter Naming Conventions](https://docs.oracle.com/javase/tutorial/java/generics/types.html)

`T` is the default type variable name suggested by the Java Language Specification. If a generic class requires more than one type variable, then it is common to use
the names `T`, `U`, `V`, and so on, or `T1`, `T2`, `T3`, and so on.

`K` and `V` are the names used in the Java Collections Framework for the key and value types when working with maps.

`E` is used in the Java Collections Framework when *defining* classes and interfaces that represent a collection of elements (such as lists and sets); for example,
inside the `List` class the element type is referred to as `E`. 

Outside of the defining class, it is also common to use `T` as the type variable name when working with a collection of elements; for example,
the `Collections` utility class uses both `List<T>` and `List<E>` when referring to a generic list.

Because a stack also represents a collection of elements, we the name `E` for the type variable name.

The type variable appears immediately after the interface or class name inside a pair of angle brackets (`<>`):

In [ ]:
public interface Stack<E> {    // type variable appears in angle brackets after the interface name

}

We can use the name `E` inside the `Stack` interface whenever we need to refer to the element type. 

Changing the original `Stack` interface to a generic interface is as simple as replacing the element type `String` with the type variable `E`:

1. change the parameter type of `push` to `E`
2. change the return type of `pop` to `E`

A fully implemented generic interface (minus the Javadoc comments) with the modifications marked by comments is shown below (run the cell to compile the interface): 

In [ ]:
public interface Stack<E> {

    public int size();

    default boolean isEmpty() {
        return this.size() == 0;
    }

    public void push(E elem);        // 1, change method parameter from String to E

    public E pop();                  // 2, change return type from String to E
}

Changing the original `ListStack` class to a generic class is as simple as replacing the element type `String` with the type variable `E`:

1. add the type variable declaration `<E>` after the class name (and generic interface name)
2. change the list element type to `E`
3. change the parameter type of `push` to `E`
4. in the `pop` method declaration, change the return type to `E`
5. in the `pop` method body, change the local variable type of `elem` to `E`

A fully implemented generic class with the modifications marked by comments is shown below (run the cell to compile the class): 

In [ ]:
import java.util.ArrayList;

public class ListStack<E> implements Stack<E> {         // 1
    
    private ArrayList<E> stack;                         // 2
    
    public ListStack() {
        this.stack = new ArrayList<>();
    }

    @Override
    public int size() {
        return this.stack.size();
    }

    @Override
    public void push(E elem) {                          // 3
        this.stack.add(elem);
    }

    @Override
    public E pop() {                                    // 4
        if (this.isEmpty()) {
            throw new RuntimeException("popped an empty stack");
        }
        E elem = this.stack.remove(this.size() - 1);    // 5
        return elem;
    }

    @Override
    public String toString() {
        StringBuilder b = new StringBuilder("Stack:");
        if (this.size() != 0) {
            for (int i = this.size() - 1; i >= 0; i--) {
                b.append('\n');
                b.append(this.stack.get(i));
            }
        }
        return b.toString();
    }
}

<a id="test_program"></a>
A short test program that creates stacks of different element types is shown below:

In [ ]:
// make sure to run the previous code cell to compile the Stack interface and ListStack class first

Stack<String> s = new ListStack<>();
s.push("a");
s.push("b");
s.push("c");
System.out.println(s);
System.out.println();

Stack<Integer> t = new ListStack<>();
t.push(1);
t.push(2);
t.push(3);
System.out.println(t);
System.out.println();

Stack<Double> u = new ListStack<>();
u.push(1.0);
u.push(2.0);
u.push(3.0);
System.out.println(u);
System.out.println();

### Type safety

Our generic `ListStack` class is type-safe: The compiler will not allow elements to pushed onto a stack where the element type is not compatible with the type variable of the stack
(run the following cell to see the compile-time error caused by trying to pushing a `double` value onto a stack of strings):

In [ ]:
// make sure to run the previous code cells to compile the Stack interface and ListStack class first

Stack<String> s = new ListStack<>();    // a stack of strings
s.push("hello");                        // ok
s.push(1.0);                            // compile-time error: 1.0 is not a String instance

Using generics, we can write type-safe methods. For example, the `sumAndClear` method from the previous notebook can be made type-safe by changing the method parameter type
to `Stack<Integer>`:

In [ ]:
public class Sum {
    
    /**
     * Returns the sum of the elements in a stack of int (or byte, char, short) or
     * zero if the stack is empty. Removes all elements from the specified stack.
     *
     * <p>
     * The returned sum may overflow the range of int.
     *
     * @param intStack a stack of Integer
     * @return the sum of the elements in the stack
     * @throws ClassCastException if the stack contains non-Integer references
     */
    public static int sumAndClear(Stack<Integer> intStack) {
        int sum = 0;
        while (intStack.size() > 0) {
            Integer elem = intStack.pop();
            sum += elem;      // no cast needed because pop returns an Integer reference
        }
        return sum;
    }
}

Passing a stack of `Integer` to the method works as expected:

In [ ]:
// make sure to run the previous code cells to compile the Stack interface, ListStack class, and Sum class first

Stack<Integer> s = new ListStack<>();    // a stack of integer
s.push(1);
s.push(2);
s.push(3);
System.out.println(Sum.sumAndClear(s));

Attempting to pass a stack of non-`Integer` values results in a compile-time error (run the following cell to view the error):

In [ ]:
// make sure to run the previous code cells to compile the Stack interface, ListStack class, and Sum class first

Stack<String> s = new ListStack<>();    // a stack of string
s.push("hello");
s.push("goodbye");
System.out.println(Sum.sumAndClear(s));

**Exercise 1** It is sometimes useful to allow the user of a stack to look at the top element of the stack without popping the element. Add a method named `peek` to the `Stack` interface that returns the
top element of the stack without popping the element.

**Exercise 2** Add a method named `contains` to the `Stack` interface that returns `true` if a specified element is somewhere in the stack.

**Exercise 3** Add a method named `reverse` to the `Stack` interface that reverses the order of the elements in this stack.

In [ ]:
// Exercises 1, 2, 3

public interface Stack<E> {

    public int size();

    default boolean isEmpty() {
        return this.size() == 0;
    }

    public void push(E elem);

    public E pop();
}

**Exercise 4** Implement the `peek` method in the `ListStack` class.

**Exercise 5** Implement the `contains` method in the `ListStack` class.

**Exercise 6** Implement the `reverse` method in the `ListStack` class.

**Exercise 7** Add a copy constructor to the `ListStack` class.

**Exercise 8** Add a constructor that initializes a `ListStack` by pushing all of the elements from a specified `Collection<E>` onto the stack (so that the last element of the collection
ends up on top of the stack).

**Exercise 9** Add a static factory method to the `ListStack` class that returns a new stack having all of the elements of a specified `ListStack` in reverse order.

In [ ]:
// Exercises 4 through 9

import java.util.ArrayList;

public class ListStack<E> implements Stack<E> {
    
    private ArrayList<E> stack;
    
    public ListStack() {
        this.stack = new ArrayList<>();
    }

    public int size() {
        return this.stack.size();
    }

    public void push(E elem) {
        this.stack.add(elem);
    }

    public E pop() {
        if (this.isEmpty()) {
            throw new RuntimeException("popped an empty stack");
        }
        E elem = this.stack.remove(this.size() - 1);
        return elem;
    }

    @Override
    public String toString() {
        StringBuilder b = new StringBuilder("Stack:");
        if (this.size() != 0) {
            for (int i = this.size() - 1; i >= 0; i--) {
                b.append('\n');
                b.append(this.stack.get(i));
            }
        }
        return b.toString();
    }
}